## LSTM FROM SCRATCH

In [135]:
import torch
import torch.nn as nn
import torch.nn.functional as F
orgi=open("orwell.txt").read().lower()
data=orgi.replace("\n"," ")
data=data.split(" ")

char2idx={char:i for i,char in enumerate(set(data))}
# char2idx['.']=0
idx2char={i:char for i,char in enumerate(set(data))}
# idx2char[0]="."
x=[]
for i in orgi.split("\n"):
    x.append([char2idx[j] for j in i.split(" ")])

X_train = []  
Y_train = []  

for sequence in x:
    for i in range(1, len(sequence)-1):
        
        X_train.append(sequence[:i])
        
        Y_train.append(sequence[i])
max_len=max([len(i) for i in X_train])
vocab_size=len(set(data))

In [136]:
char2idx

{'reaching': 0,
 'five,': 1,
 'seen': 2,
 'dressed': 3,
 'clothes': 4,
 'smokeless': 5,
 'cruelty': 6,
 'light,': 7,
 'coat': 8,
 'per': 9,
 'shoving': 10,
 'wandering': 11,
 'captivity.': 12,
 'cut': 13,
 'stone-floored': 14,
 'smoking,': 15,
 'medical': 16,
 'supperâ€”half': 17,
 'his': 18,
 'happens': 19,
 'forty-nine': 20,
 'beer': 21,
 'hiding': 22,
 'twenty': 23,
 'went': 24,
 'honour': 25,
 'black': 26,
 'died': 27,
 'shivering': 28,
 'wen': 29,
 'astonishing;': 30,
 "mate,'": 31,
 'garments,': 32,
 'cold': 33,
 'marching': 34,
 'stone': 35,
 'slices': 36,
 'tables.': 37,
 'cook': 38,
 'managed': 39,
 'first': 40,
 'panacea': 41,
 'hard': 42,
 "major's": 43,
 'underwear': 44,
 'overnight,': 45,
 'worse': 46,
 'herculean': 47,
 'bags': 48,
 'souls.': 49,
 'each': 50,
 '11.': 51,
 'appeared': 52,
 'us.': 53,
 'pot-bellied,': 54,
 'benches,': 55,
 'smelt': 56,
 'lazarus': 57,
 'never': 58,
 'water': 59,
 'elbow': 60,
 'buried': 61,
 'were': 62,
 'misconducted': 63,
 'passage.': 64,

In [137]:
def pre_pad_sequences_pytorch(sequences, max_len):
    padded_sequences = []
    
    for seq in sequences:
        # If the sequence is shorter than max_len, pad with zeros at the beginning
        if len(seq) < max_len:
            padded_seq = [0] * (max_len - len(seq)) + seq  # Pre-padding with 0
        # If the sequence is longer than max_len, truncate it
        else:
            padded_seq = seq[-max_len:]  # Keep the last max_len elements
        padded_sequences.append(padded_seq)
    
    return torch.tensor(padded_sequences)

In [138]:
X_train_padded = pre_pad_sequences_pytorch(X_train, max_len)
X_train_padded=X_train_padded.unsqueeze(-1)
Y_train=torch.tensor(Y_train)

In [139]:
max_len

213

In [144]:

criterion = nn.CrossEntropyLoss()

class LSTM:
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        self.hidden_size = hidden_size
        self.embedding = torch.randn(vocab_size, embedding_dim, requires_grad=True)

        # Initialize weights with requires_grad=True
        self.Wf = torch.randn(hidden_size, embedding_dim + hidden_size, requires_grad=True)
        self.bf = torch.zeros(hidden_size, requires_grad=True)
        
        self.Wi = torch.randn(hidden_size, embedding_dim + hidden_size, requires_grad=True)
        self.bi = torch.zeros(hidden_size, requires_grad=True)
        
        self.Wc = torch.randn(hidden_size, embedding_dim + hidden_size, requires_grad=True)
        self.bc = torch.zeros(hidden_size, requires_grad=True)
        
        self.Wo = torch.randn(hidden_size, embedding_dim + hidden_size, requires_grad=True)
        self.bo = torch.zeros(hidden_size, requires_grad=True)

        self.Wv = torch.randn(output_size, hidden_size, requires_grad=True)
        self.bv = torch.zeros(output_size, requires_grad=True)
        self._initialize_weights()
    
    def _initialize_weights(self):
        nn.init.xavier_uniform_(self.Wf)
        nn.init.xavier_uniform_(self.Wi)
        nn.init.xavier_uniform_(self.Wc)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.Wv)


    def parameters(self):
        # Return a list of all parameters (weights and biases) in the model
        return [self.Wf, self.bf, self.Wi, self.bi, self.Wc, self.bc, self.Wo, self.bo, self.Wv, self.bv, self.embedding]

    def forward(self, x, init_states=None):
    # Apply embedding layer to input indices
        # print(f"X shape coming in model.forward{x.shape}")
        x=x.squeeze(dim=-1)
        # print(x.shape)
        x = self.embedding[x] 
        # print(f"X shape coming after embedding{x.shape}")
        
        # print(x.shape)
        # Shape: (batch_size, seq_len, embedding_dim)
        batch_size, seq_len, _ = x.size()
    
    # Initialize h_t and c_t if init_states is None
        if init_states is None:
            h_t = torch.zeros(batch_size, self.hidden_size, requires_grad=True)
            c_t = torch.zeros(batch_size, self.hidden_size, requires_grad=True)
        else:
            h_t, c_t = init_states
        
        outputs = []
        for t in range(seq_len):
            x_t = x[:, t, :]  # Shape: (batch_size, embedding_dim)
            
            Z_t = torch.cat([x_t, h_t], dim=1)  # Shape: (batch_size, embedding_dim + hidden_size)
            # print(Z_t.shape)
            # break
            # Forget gate
            ft = torch.sigmoid(Z_t @ self.Wf.t() + self.bf)
            # Input gate
            it = torch.sigmoid(Z_t @ self.Wi.t() + self.bi)
            # Candidate cell state
            can = torch.tanh(Z_t @ self.Wc.t() + self.bc)
            # Output gate
            ot = torch.sigmoid(Z_t @ self.Wo.t() + self.bo)
        
            
            c_t = c_t * ft + can * it
            
            h_t = ot * torch.tanh(c_t)
        
            # Compute output for current time step
            y_t = h_t @ self.Wv.t() + self.bv
            # outputs.append(y_t)
        
        # Stack outputs across the time dimension (seq_len)
        # outputs = torch.stack(outputs, dim=1)
        return y_t, (h_t, c_t)



In [141]:
model = LSTM(vocab_size=vocab_size, embedding_dim=10, hidden_size=128, output_size=vocab_size)
params = model.parameters()
optimizer = torch.optim.Adam(params, lr=0.005)

In [143]:
hidden_state = None  # Initialize hidden state
for _ in range(200):
    # Sample a batch
    batch_indices = torch.randint(0, X_train_padded.shape[0], (128,))
    x_train = X_train_padded[batch_indices]  # Shape: (batch_size, seq_len)
    
    # Forward pass
    # print(f"x_train passed into forward{x_train.shape}")
    outputs,hidden_state = model.forward(x_train, init_states=hidden_state)
    # print(outputs.shape)
    h_t, c_t = hidden_state
    hidden_state = (h_t.detach(), c_t.detach())  # Detach hidden state for next batch

    # Compute loss
    y_batch = Y_train[batch_indices]  # Shape: (batch_size, seq_len, vocab_size)
    loss = criterion(outputs, y_batch)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss
    print(loss)


tensor(6.1963, grad_fn=<NllLossBackward0>)
tensor(6.1273, grad_fn=<NllLossBackward0>)
tensor(6.2829, grad_fn=<NllLossBackward0>)
tensor(6.2121, grad_fn=<NllLossBackward0>)
tensor(6.3325, grad_fn=<NllLossBackward0>)
tensor(6.1679, grad_fn=<NllLossBackward0>)
tensor(6.3374, grad_fn=<NllLossBackward0>)
tensor(6.1030, grad_fn=<NllLossBackward0>)
tensor(6.4398, grad_fn=<NllLossBackward0>)
tensor(6.0408, grad_fn=<NllLossBackward0>)
tensor(6.3122, grad_fn=<NllLossBackward0>)
tensor(6.4605, grad_fn=<NllLossBackward0>)
tensor(6.1222, grad_fn=<NllLossBackward0>)
tensor(6.1336, grad_fn=<NllLossBackward0>)
tensor(6.2812, grad_fn=<NllLossBackward0>)
tensor(5.9523, grad_fn=<NllLossBackward0>)
tensor(5.8275, grad_fn=<NllLossBackward0>)
tensor(6.3046, grad_fn=<NllLossBackward0>)
tensor(6.1705, grad_fn=<NllLossBackward0>)
tensor(6.0598, grad_fn=<NllLossBackward0>)
tensor(6.1097, grad_fn=<NllLossBackward0>)
tensor(6.0187, grad_fn=<NllLossBackward0>)
tensor(5.7385, grad_fn=<NllLossBackward0>)
tensor(6.13

In [146]:
def generate_sequence(model, seed_string, char2idx, idx2char, sequence_length, max_len=55):
    """
    Generates a sequence from the trained LSTM model given a seed string.

    Args:
        model: Trained LSTM model.
        seed_string: The initial string to seed the sequence generation.
        char2idx: Dictionary mapping characters to indices.
        idx2char: Dictionary mapping indices to characters.
        sequence_length: Number of tokens to generate.
        max_len: Maximum length of input sequence for the model.

    Returns:
        Generated sequence as a string.
    """
    # model.eval()  # Set the model to evaluation mode
    
    # Convert seed string to sequence of indices
    seed_indices = [char2idx[word] for word in seed_string.split(" ") if word in char2idx]
    seed_tensor = torch.tensor(seed_indices).unsqueeze(0)  # Shape: (1, seq_len)
    
    generated_indices = seed_indices[:]  # Start with the seed sequence
    hidden_state = None  # Initialize hidden state

    for _ in range(sequence_length):
        # Pre-pad the input sequence to match the model's expected input size
        padded_input = pre_pad_sequences_pytorch([generated_indices], max_len).unsqueeze(-1)
        # print(padded_input.shape)
        # Get the model output and hidden state
        output, (hidden_state) = model.forward(padded_input, hidden_state)
        # print(output.shape)
        
        # Take the output corresponding to the last token
        next_token_logits = output  # Shape: (1, vocab_size)
        
        # Use softmax to get probabilities and sample the next token
        next_token_prob = torch.softmax(next_token_logits, dim=-1)
        # next_token_idx = torch.multinomial(next_token_prob, num_samples=1).item()
        next_token_idx=torch.argmax(next_token_prob).item()
        
        # Append the predicted token to the sequence
        generated_indices.append(next_token_idx)
    
    # Convert indices back to characters
    generated_words = [idx2char[idx] for idx in generated_indices]
    return " ".join(generated_words)


# Example usage:
seed_string = "scum"
sequence_length = 40
generated_text = generate_sequence(model, seed_string, char2idx, idx2char, sequence_length)
print("Generated Text:")
print(generated_text)


Generated Text:
scum hurried our ankles the dining-room and of us and to the waist of christ, mate,' the food was the may set and we had eightpence in our cigarettes with tobacco to the spike and then and him the stuff and


In [ ]:
Generated Text:
scum was set forty-eight men one men among us, forty-eight and left cold, was worse tub the it tobacco and its of the would washing-up, and both. was astonishing; One me two cold There It was off and the prison The room. was a set in a lime-washed, tramp pot-bellied, degenerate curs we looked. Shock heads, hairy, crumpled faces, hollow chests, flat feet, sagging muscles—every kind of malformation and physical rottenness were there. All were flabby and discoloured, as all tramps are under their deceptive sunburn. Two or three figures wen there stay ineradicably in my mind. Old 'Daddy', aged seventy-four,